<a href="https://colab.research.google.com/github/gerardonunez-nyt/apps-script-samples/blob/master/Storytelling_Testing_Semi_Automation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A/B Testing Data Validation

In [ ]:
# Authenticate, because access to BigQuery is not publicly available.
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

# Select the project you'll be using. This will be attached to your email with the Oauth2 authentication protocol.
project_id = 'nyt-bigquery-beta-workspace'

# Import the package
from google.cloud import bigquery

# Create a client object, with your credentials and directed at your project.
client = bigquery.Client(project=project_id)

# Imports
import pandas as pd
pd.set_option('max_rows', 9999)

Authenticated


In [ ]:
# Define variables
experiment_data = {
  'test': ''' 'DFP_live_0722' ''',
  'start_date': ''' '2022-08-09' ''',
  'end_date': ''' '2022-08-09' ''',
  'validation_table': 'validation_DFP_live_0722'
}

In [ ]:
# Collect all exposures from et.page

# We need this logic to be reusable (view)
# We need this logic to run for multiple parallel tests (parameterize with a list)
# We need this logic to run daily for multiple tests (Bisque / dbt)

collect_exposures = """
CREATE OR REPLACE TABLE `nyt-bigquery-beta-workspace.story.validation_table` AS(

SELECT
  source_app,
  DATE(_pt) AS date,
  ab.test,
  ab.variant,
  pageview_id,
  agent_id,
  combined_regi_id,
  coalesce(safe_cast(combined_regi_id as string), agent_id) as user_id, 
  timestamps.started_dt_nyct as pv_started_dt_nyct,
  ab.dt_nyct as expose_dt_nyct,
  CASE
    WHEN agent.user_agent LIKE '%nytios%' THEN 'nyt_ios'
    WHEN agent.user_agent LIKE '%nytiphone%' THEN 'nyt_ios'
    WHEN agent.user_agent LIKE '%nytipad%' THEN 'nyt_ios'
    WHEN LOWER(agent.user_agent) LIKE '%nyt_android%' THEN 'nyt_android'
    WHEN p.source_app = 'nyt-vi' AND agent.device.is_computer THEN 'desktop'
    WHEN p.source_app = 'nyt-vi' AND (agent.device.is_mobile OR agent.device.is_tablet) THEN 'mobile_web'
    ELSE NULL
  END as platform
FROM
  `nyt-eventtracker-prd.et.page` p,
  UNNEST(ab_exposes) AS ab
WHERE
  DATE(_pt) BETWEEN """ + experiment_data['start_date'] + """
  AND """ + experiment_data['end_date'] + """
  AND ab.test = """ + experiment_data['test'] + """ ) """

client.query(collect_exposures)

# Check for leakage

Agents who are exposed to multiple variants.

In [ ]:
# We want to use the leakage result to filter
# We want to calculate leakage % and display in Colab
# We don't want to store this info as a table

leakage_from_exposures = """
CREATE TEMP TABLE leakage_exposures AS(

SELECT
  agent_id,
  count(distinct variant) as variant_exposure,
  case when count(distinct variant) > 1 then 1
  else 0
  end as is_leakage
FROM
  `nyt-bigquery-beta-workspace.story.validation_table`
GROUP BY
  1);
"""

leakage_pct = leakage_from_exposures + """

SELECT
  COUNT(DISTINCT CASE WHEN is_leakage = 1 THEN agent_id END) leakaged_agents,
  COUNT(DISTINCT CASE WHEN is_leakage = 0 THEN agent_id END) non_leaked_agents,
  SAFE_DIVIDE(
    COUNT(DISTINCT CASE WHEN is_leakage = 1 THEN agent_id END),
    COUNT(DISTINCT agent_id)
             ) pct_leakage
FROM
  leakage_exposures
"""

leakage_pct_data = client.query(leakage_pct).to_dataframe()

leakage_pct_data

## Remove leakage from exposures data

In [ ]:
remove_leakage = leakage_from_exposures + """

DELETE FROM `nyt-bigquery-beta-workspace.story.validation_table`
WHERE
agent_id IN (SELECT DISTINCT agent_id FROM leakage_exposures WHERE is_leakage = 1)

"""

client.query(remove_leakage)

# Check proportions of variants
Overall and by platform.

In [ ]:
variant_check = """
SELECT
  variant,
  COUNT(DISTINCT agent_id) as agents, 
  ROUND(COUNT(DISTINCT agent_id) / SUM(COUNT(DISTINCT agent_id)) OVER(), 4) percent_total
FROM
  `nyt-bigquery-beta-workspace.story.validation_table`
GROUP BY
  1
ORDER BY 
  1
"""

variant_check_data = client.query(variant_check).to_dataframe()

variant_check_data

In [ ]:
platform_check = """
SELECT
  variant,
  platform,
  COUNT(DISTINCT agent_id) agents,
  COUNT(DISTINCT pageview_id) pageviews,
  COUNT(1) exposures
FROM
  `nyt-bigquery-beta-workspace.story.validation_table`
GROUP BY
  1,2
ORDER BY
  1,2
"""

platform_check_data = client.query(platform_check).to_dataframe()

platform_check_data

## Check % of test pageviews over total pvs
Not needed for every test

In [ ]:
test_pvs_check = """
WITH 
expose_data AS (
SELECT date, COUNT(DISTINCT pageview_id) expose_pageviews
FROM `nyt-bigquery-beta-workspace.story.validation_table`
GROUP BY 1),

pv_data AS (
SELECT 
  DATE( _PARTITIONTIME) as date, 
  SUM(page_views) total_live_parent_pageviews
FROM `nyt-bigquery-beta-workspace.audience_data.behavior_counts`
INNER JOIN `nyt-bigquery-beta-workspace.dig_pubp.content_master_live_parents` USING(uri)
WHERE 
  DATE(_PARTITIONTIME) BETWEEN """ + experiment_data['start_date'] + """
  AND """ + experiment_data['end_date'] + """
GROUP BY
  1)

SELECT date, pv_data.total_live_parent_pageviews, expose_data.expose_pageviews, 
  ROUND((expose_data.expose_pageviews / pv_data.total_live_parent_pageviews), 4) AS percent_test_over_pvs
FROM pv_data
LEFT JOIN expose_data USING(date)
ORDER BY 1
"""

test_pvs_check_data = client.query(test_pvs_check).to_dataframe()

test_pvs_check_data